# Referências da Documentação AWS:

- https://docs.aws.amazon.com/nova/latest/userguide/prompting-structured-output.html

# Dependências

Importação das bibliotecas necessárias para interagir com AWS Bedrock e manipular dados JSON.

In [ ]:
import boto3
import json
import os

# Constantes

In [ ]:
# Pasta local para salvar respostas das chamadas AWS
OUTPUT_FOLDER = "respostas"

# ID do modelo Amazon Nova Lite (modelo de linguagem da AWS)
MODEL_ID = 'amazon.nova-lite-v1:0'

# Configuração de diretórios

In [ ]:
os.makedirs(OUTPUT_FOLDER, exist_ok=True)

# Configurações do AWS BEDROCK

In [ ]:
# Inicializa o cliente do Bedrock Runtime para fazer chamadas à API
bedrock_runtime_client = boto3.client('bedrock-runtime')

# Relembrando a Estrutura Básica

Exemplo simples de chamada ao Bedrock para relembrar a estrutura básica da API `converse`.

In [ ]:
response = bedrock_runtime_client.converse(
    modelId=MODEL_ID,
    messages=[{
        "role": "user",
         "content": [{"text": "Qual é a capital do Brasil?"}]
    }],
    inferenceConfig={"maxTokens": 400, "temperature": 0.5}
)

# Salvar os resultados em um arquivo JSON
with open(f"{OUTPUT_FOLDER}/response_bedrock_runtime_client_01.json", 'w', encoding='utf-8') as json_file:
    json.dump(response, json_file, ensure_ascii=False, indent=4, default=str)

resultado = response['output']['message']['content'][0]['text']
print(resultado)

# Método 1: Prompt Engineering

Este método é o mais intuitivo, mas apresenta riscos: a IA pode gerar texto com variações ou simplesmente não respeitar o prompt. Os dois exemplos abaixo são muito similares, mas demonstram que pequenas variações no prompt têm grande impacto na saída do modelo.

In [ ]:
# Exemplo de prompt que gera uma saída pronta para uso
response = bedrock_runtime_client.converse(
    modelId=MODEL_ID,
    messages=[{
        "role": "user",
        "content": [{
            "text": """
            Crie dados de uma loja online com produtos e clientes.
            Responda APENAS um objeto semelhante a um JSON seguindo este formato:
            {
                "loja": {
                    "nome": "string",
                    "produtos": [
                        {"id": number, "nome": "string", "preco": number, "categoria": "string"}
                    ],
                    "clientes": [
                        {"id": number, "nome": "string", "email": "string", "cidade": "string"}
                    ]
                }
            }
            
          
            """
        }]
    }],
    inferenceConfig={"maxTokens": 800, "temperature": 0}
)

resultado = response['output']['message']['content'][0]['text']
print(resultado)

In [ ]:
# Salvar os resultados em um arquivo JSON
with open(f"{OUTPUT_FOLDER}/response_bedrock_runtime_client_02.json", 'w', encoding='utf-8') as json_file:
    json.dump(response, json_file, ensure_ascii=False, indent=4, default=str)

In [ ]:
# Exemplo de prompt que gera uma saída estilo markdown
response = bedrock_runtime_client.converse(
    modelId=MODEL_ID,
    messages=[{
        "role": "user",
        "content": [{
            "text": """
            Crie dados de uma loja online com produtos e clientes.
            Responda APENAS com JSON válido seguindo este formato:
            {
                "loja": {
                    "nome": "string",
                    "produtos": [
                        {"id": number, "nome": "string", "preco": number, "categoria": "string"}
                    ],
                    "clientes": [
                        {"id": number, "nome": "string", "email": "string", "cidade": "string"}
                    ]
                }
            }
            """
        }]
    }],
    inferenceConfig={"maxTokens": 800, "temperature": 0}
)
resultado = response['output']['message']['content'][0]['text']
print(resultado)

In [ ]:
# Para lidar com a possibilidade de markdown, podemos limpar a marcação se presente
resultado = resultado.replace('```json', '').replace('```', '').strip()

# Validar JSON
try:
    data = json.loads(resultado)
    print("\n✅ JSON válido!")
    with open(f'{OUTPUT_FOLDER}/teste_metodo_01_prompt_engineering.json', 'w') as f:
        json.dump(data, f, indent=2, ensure_ascii=False)
except json.JSONDecodeError as e:
    print(f"\n❌ JSON inválido: {e}")

# Método 2: Prefilling

Método baseado em pré-preenchimento da resposta. Você mostra para a IA como deseja que a resposta inicie. Neste caso, vamos iniciar com a estrutura de um JSON e esperar que ela prossiga com o formato correto. Este método pode ser combinado com o anterior, mas para este tutorial usaremos um prompt mais genérico para confirmar que o prefill está funcionando (e que não é resultado do prompt engineering).

In [ ]:
# Exemplo sem prefilling
response = bedrock_runtime_client.converse(
    modelId=MODEL_ID,
    messages=[
        {
            "role": "user",
            "content": [{"text": "Liste 5 países com suas capitais e população em formato JSON"}]
        }
    ],
    inferenceConfig={
        "maxTokens": 600,
        "temperature": 0
    }
)
resultado = response['output']['message']['content'][0]['text']
print(resultado)

In [ ]:
# Exemplo com prefilling da abertura do json "{"
response = bedrock_runtime_client.converse(
    modelId=MODEL_ID,
    messages=[
        {
            "role": "user",
            "content": [{"text": "Liste 5 países com suas capitais e população em formato JSON"}]
        },
        {
            "role": "assistant",
            "content": [{"text": "{"}]  # Prefilling - A IA assume que este início de string já está presente na saída, forçando o início do JSON
        }
    ],
    inferenceConfig={
        "maxTokens": 600,
        "temperature": 0
    }
)
resultado = response['output']['message']['content'][0]['text']
print(resultado)

Observe que a resposta parece um JSON que faltou o "{" inicial. Isso acontece porque é assumido que já foi fornecido esse caractere. Agora vamos testar a validação:

In [ ]:
# Salvar os resultados em um arquivo JSON
with open(f"{OUTPUT_FOLDER}/response_bedrock_runtime_client_03.json", 'w', encoding='utf-8') as json_file:
    json.dump(response, json_file, ensure_ascii=False, indent=4, default=str)

In [ ]:
# Reconstrói o JSON completo adicionando o "{" inicial
resultado = "{"  + response['output']['message']['content'][0]['text']
print("Método 2 - Prefilling:")
print(resultado)

In [ ]:
# Validar JSON
try:
    data = json.loads(resultado)
    print("\n✅ JSON válido!")
    with open(f'{OUTPUT_FOLDER}/teste_metodo_02_prefilling.json', 'w') as f:
        json.dump(data, f, indent=2, ensure_ascii=False)
except json.JSONDecodeError as e:
    print(f"\n❌ JSON inválido: {e}")

# Método 3: Tool Use (Amazon Nova)

In [ ]:
# Schema JSON Schema para definir a estrutura esperada no Tool Use
# Veja mais detalhes em https://docs.aws.amazon.com/nova/latest/userguide/prompting-structured-output.html
schema = {
    "json": {
        "type": "object",
        "properties": {
            "produtos": {
                "type": "array",
                "items": {
                    "type": "object",
                    "properties": {
                        "nome": {"type": "string"},
                        "preco": {"type": "number"},
                        "categoria": {"type": "string"}
                    },
                    "required": ["nome", "preco", "categoria"]
                }
            }
        },
        "required": ["produtos"]
    }
}

response = bedrock_runtime_client.converse(
    modelId=MODEL_ID,
    messages=[{
        "role": "user",
        "content": [{"text": "Crie 3 produtos eletrônicos. Use a ferramenta gerar_produtos."}]
    }],
    toolConfig={
        "tools": [{
            "toolSpec": {
                "name": "gerar_produtos",
                "description": "Gera lista de produtos",
                "inputSchema": schema
            }
        }],
        "toolChoice": {"tool": {"name": "gerar_produtos"}}
    },
    inferenceConfig={"maxTokens": 500, "temperature": 0}
)

In [ ]:
# Salvar os resultados em um arquivo JSON
with open(f"{OUTPUT_FOLDER}/response_bedrock_runtime_client_04.json", 'w', encoding='utf-8') as json_file:
    json.dump(response, json_file, ensure_ascii=False, indent=4, default=str)

In [ ]:
resultado = response['output']['message']['content'][0]['toolUse']['input']
print("Método 3 - Tool Use:")
print(json.dumps(resultado, indent=2, ensure_ascii=False))

In [ ]:
# Validar JSON - Note que a saída já é um dicionário Python válido
try:
    data = resultado # Aqui observe que a saída já é um dicionário Python válido
    print("\n✅ JSON válido!")
    with open(f'{OUTPUT_FOLDER}/teste_metodo_03_tool_use.json', 'w') as f:
        json.dump(data, f, indent=2, ensure_ascii=False)
except json.JSONDecodeError as e:
    print(f"\n❌ JSON inválido: {e}")